## Install MQTT client

## Updated Live Emotion Recognition with MQTT

In [3]:
import cv2
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import paho.mqtt.client as mqtt
import time
import json

# --- Config ---
num_classes = 7
save_path = r"C:\Users\HWA\Desktop\AI Project UV\src\models\Final_ModelV2.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- MQTT Config ---
BROKER_IP = "192.168.43.1"   # 🔹 replace with your phone's broker IP
BROKER_PORT = 1883
TOPIC = "emotion/detection"

client = mqtt.Client()
client.connect(BROKER_IP, BROKER_PORT, 60)
client.loop_start()  # start background loop

# --- Transform ---
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# --- Model ---
model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load(save_path, map_location=device))
model = model.to(device)
model.eval()

# --- Emotion labels ---
emotions = {
    0:"Angry", 1:"Disgust", 2:"Fear",
    3:"Happy", 4:"Neutral", 5:"Sad", 6:"Surprised"
}

# --- Load Haar Cascade for face detection ---
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# --- Webcam ---
cap = cv2.VideoCapture(1)  # 0 = built-in, 1 = external
if not cap.isOpened():
    raise IOError(" Cannot open webcam")

# --- Smart publishing variables ---
last_publish_time = 0
publish_interval = 1.0   # seconds
last_status = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)

    current_time = time.time()
    status = "No Face"  # default if no face detected

    if len(faces) > 0:
        # Only handle the first detected face
        (x, y, w, h) = faces[0]
        roi_color = frame[y:y+h, x:x+w]

        # Convert to PIL image
        img_pil = Image.fromarray(cv2.cvtColor(roi_color, cv2.COLOR_BGR2RGB))
        img_tensor = transform(img_pil).unsqueeze(0).to(device)

        # Predict emotion
        with torch.no_grad():
            outputs = model(img_tensor)
            _, predicted = torch.max(outputs, 1)
            status = emotions[predicted.item()]

        # Draw rectangle and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, status, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.9, (0, 0, 255), 2, cv2.LINE_AA)

    # --- Smart publishing: only if changed or after interval ---
    if status != last_status or (current_time - last_publish_time >= publish_interval):
        payload = json.dumps({"emotion": status})
        client.publish(TOPIC, payload)
        print(f"Published: {payload}")  # debug print
        last_publish_time = current_time
        last_status = status

    cv2.imshow("Live Emotion Recognition", frame)

    if cv2.waitKey(2) & 0xFF == ord("q"):
        break

# --- Cleanup ---
cap.release()
cv2.destroyAllWindows()
client.loop_stop()
client.disconnect()


C:\Users\HWA\AppData\Local\Temp\ipykernel_21944\3341566119.py:20: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Published: {"emotion": "No Face"}
Published: {"emotion": "No Face"}
Published: {"emotion": "No Face"}
Published: {"emotion": "No Face"}
Published: {"emotion": "No Face"}
Published: {"emotion": "Neutral"}
Published: {"emotion": "No Face"}
Published: {"emotion": "Sad"}
Published: {"emotion": "No Face"}
Published: {"emotion": "No Face"}
Published: {"emotion": "Fear"}
Published: {"emotion": "No Face"}
Published: {"emotion": "Fear"}
Published: {"emotion": "No Face"}
Published: {"emotion": "Angry"}
Published: {"emotion": "Sad"}
Published: {"emotion": "No Face"}
Published: {"emotion": "Neutral"}
Published: {"emotion": "No Face"}
Published: {"emotion": "Sad"}
Published: {"emotion": "Neutral"}
Published: {"emotion": "Angry"}
Published: {"emotion": "Sad"}
Published: {"emotion": "Surprised"}
Published: {"emotion": "Sad"}
Published: {"emotion": "Fear"}
Published: {"emotion": "Neutral"}
Published: {"emotion": "Sad"}
Published: {"emotion": "Neutral"}
Published: {"emotion": "Neutral"}
Published: {"em

<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>